In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf

In [5]:
file_path = "/content/data_balita.csv"
data = pd.read_csv(file_path)

In [6]:
# Exploratory Data Analysis (EDA)
print("Dataset Overview:")
print(data.head())
print("\nMissing Values:")
print(data.isnull().sum())

Dataset Overview:
   Umur (bulan) Jenis Kelamin  Tinggi Badan (cm)       Status Gizi
0             0     laki-laki          44.591973           stunted
1             0     laki-laki          56.705203            tinggi
2             0     laki-laki          46.863358            normal
3             0     laki-laki          47.508026            normal
4             0     laki-laki          42.743494  severely stunted

Missing Values:
Umur (bulan)         0
Jenis Kelamin        0
Tinggi Badan (cm)    0
Status Gizi          0
dtype: int64


In [7]:
data.rename(columns={
    "Umur (bulan)": "umur_bulan",
    "Jenis Kelamin": "jenis_kelamin",
    "Tinggi Badan (cm)": "tinggi",
    "Status Gizi": "status_gizi"
}, inplace=True)

# Periksa apakah kolom telah diubah
print("Renamed Columns:")
print(data.columns)

Renamed Columns:
Index(['umur_bulan', 'jenis_kelamin', 'tinggi', 'status_gizi'], dtype='object')


In [8]:
# Handle missing values
data = data.dropna()

In [9]:
# Encode categorical features
categorical_columns = data.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

In [10]:
target = pd.get_dummies(data["status_gizi"], prefix="status_gizi")
data = pd.concat([data.drop(columns=["status_gizi"]), target], axis=1)

In [11]:
data.head()

,umur_bulan,jenis_kelamin,tinggi,status_gizi_0,status_gizi_1,status_gizi_2,status_gizi_3
0,0,0,44.591973,False,False,True,False
1,0,0,56.705203,False,False,False,True
2,0,0,46.863358,True,False,False,False
3,0,0,47.508026,True,False,False,False
4,0,0,42.743494,False,True,False,False


In [54]:
# Split features and target
X = data.drop(columns=target.columns)  # Semua kolom kecuali target
y = data[target.columns]  # Kolom target yang telah di-one-hot

In [52]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Mengatur input shape di sini
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # For multi-class classification
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Save the trained model
model.save("model_balita.h5")

Epoch 1/5
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8428 - loss: 0.4241 - val_accuracy: 0.9627 - val_loss: 0.1054
Epoch 2/5
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9639 - loss: 0.0991 - val_accuracy: 0.9678 - val_loss: 0.0855
Epoch 3/5
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9700 - loss: 0.0763 - val_accuracy: 0.9735 - val_loss: 0.0729
Epoch 4/5
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9742 - loss: 0.0643 - val_accuracy: 0.9704 - val_loss: 0.0699
Epoch 5/5
2420/2420 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9745 - loss: 0.0623 - val_accuracy: 0.9678 - val_loss: 0.0659
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9684 - loss: 0.0657


Test Loss: 0.064586341381073
Test Accuracy: 0.9685124158859253


In [53]:
# Load trained model
model = tf.keras.models.load_model("model_balita.h5")
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 128)                 │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,030 (35.28 KB)

 Trainable params: 9,028 (35.27 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [63]:
# Prepare input data
input_data = {
    "umur_bulan": 8,
    "jenis_kelamin": "perempuan",
    "tinggi": 72
}

# Convert input data to DataFrame
input_df = pd.DataFrame([input_data])

# Encode categorical features
input_df['jenis_kelamin'] = label_encoders['jenis_kelamin'].transform(input_df['jenis_kelamin'])

# Ensure the order of columns matches the original training data
input_df = input_df[["umur_bulan", "jenis_kelamin", "tinggi"]]

# Scale the input data
input_array_scaled = scaler.transform(input_df)

In [64]:
# Mapping indeks ke label deskriptif
class_labels = ["Normal", "Severaly Stunting", "Stunting", "Tinggi"]

# Predict
prediction = model.predict(input_array_scaled)

# Interpret prediction
predicted_class_index = np.argmax(prediction)  # Indeks kelas dengan probabilitas tertinggi
predicted_class_label = class_labels[predicted_class_index]  # Label deskriptif

# Print results
print(f"Prediction (raw probabilities): {prediction[0]}")
print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted class: {predicted_class_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction (raw probabilities): [9.9998868e-01 1.2061245e-30 7.7469607e-11 1.1321887e-05]
Predicted class index: 0
Predicted class: Normal
